In [1]:
import Arm_Lib
import numpy as np
import cv2
import time
import math

In [2]:
Arm = Arm_Lib.Arm_Device()

def arm_move16(a,b,arr,t):
    Arm.Arm_serial_servo_write6(a,arr[0],arr[1],arr[2],arr[3],b,t*1000)
    time.sleep(t+0.5)

def get_arm_pos():
    ret = []
    for i in range(6):
        ri = Arm.Arm_serial_servo_read(i+1)
        ret.append(ri)
    return ret


ARM_CLAMP_LOOSE = 31
ARM_CLAMP_TIGHT = 83

ARM_ANGLE_MID = 90

ARM_down_toclamp = [  30, 23, 134, 89]
ARM_up_toclamp = [131, 11, 42, 89]

ARM_pour_start =[59, 0, 120, 89]

ARM_POUR_L = 180
ARM_POUR_M = 90
ARM_POUR_R = 0

ARM_ANGLE_MOVEMENT = 15


def arm_clamp(enable):
    target = 0
    if not enable:
        target = ARM_CLAMP_LOOSE
    else:
        target = ARM_CLAMP_TIGHT
    Arm.Arm_serial_servo_write(6,target,400)
    time.sleep(0.5)
    
def arm_moveto_angle(angle):
    Arm.Arm_serial_servo_write(1,angle,1000)
    time.sleep(1.5)

def arm_pour(enable, lr):
    target = 0
    if enable:
        if lr == 'left':
            target = ARM_POUR_L
        else:
            target = ARM_POUR_R
    else:
        target = ARM_POUR_M

    Arm.Arm_serial_servo_write(5,target,500)
    time.sleep(0.5)
        

def arm_reset():
    target = ARM_up_toclamp
    arm_move16(ARM_ANGLE_MID,ARM_CLAMP_LOOSE,target,2)

    

In [33]:
arm_reset()

In [4]:
def arm_grip_and_up(angle):
    arm_moveto_angle(angle)
    arm_move16(angle,ARM_CLAMP_LOOSE,ARM_up_toclamp,1)
    arm_move16(angle,ARM_CLAMP_LOOSE,ARM_down_toclamp,2)
    arm_clamp(True)
    arm_move16(angle,ARM_CLAMP_TIGHT,ARM_up_toclamp,2)

def arm_grip_down(angle):
    arm_moveto_angle(angle)
    arm_move16(angle,ARM_CLAMP_TIGHT, ARM_down_toclamp,2)
    arm_clamp(False)
    arm_move16(angle,ARM_CLAMP_LOOSE,ARM_up_toclamp,2)

def arm_pour_down(angle, lr):
    arm_moveto_angle(angle)
    tangle = angle
    if lr == 'left':
        tangle+=ARM_ANGLE_MOVEMENT
    else:
        tangle-=ARM_ANGLE_MOVEMENT
    arm_move16(angle, ARM_CLAMP_TIGHT, ARM_pour_start,2)
    arm_moveto_angle(tangle)
    arm_pour(True,lr)
    time.sleep(0.7)
    arm_pour(False,lr)
    arm_moveto_angle(angle)
    arm_move16(angle,ARM_CLAMP_TIGHT,ARM_up_toclamp,2)


In [5]:
def arm_grip_and_pour(f,t,lr):
    arm_grip_and_up(f)
    arm_pour_down(t,lr)
    arm_grip_down(f)

In [51]:
arm_grip_and_pour(90,60,'left')

In [6]:
def get_color(img):
    H = []
    color_name = {}
    
    img = cv2.resize(img, (640, 480), )
    HSV = cv2.cvtColor(img, cv2.COLOR_BGR2HSV)
    cv2.rectangle(img, (280, 180), (360, 260), (0, 255, 0), 2)
  
    for i in range(280, 360):
        for j in range(180, 260): 
            if HSV[j,i][1]>100 and HSV[j,i][2]>60:
                H.append(HSV[j, i][0])

    if(len(H)==0):
        return img, color_name
    
    H_min = min(H);H_max = max(H)
    # print(H_min,H_max)
    
    # 颜色判断
    if H_min >= 0 and H_max <= 10 or H_min >= 156 and H_max <= 180: color_name['name'] = 'red'
    elif H_min >= 25 and H_max <= 32: color_name['name'] = 'yellow'
    elif H_min >= 70 and H_max <= 88: color_name['name'] = 'green'
    elif H_min >= 90 and H_max <= 105: color_name['name'] = 'blue'
    return img, color_name

In [55]:
cap = cv2.VideoCapture(0)
cap.set(3, 480)
cap.set(4, 320)
cap.set(5, 15)  #设置帧率
cap.set(cv2.CAP_PROP_FOURCC, cv2.VideoWriter.fourcc('M', 'J', 'P', 'G'))
cap.set(cv2.CAP_PROP_BUFFERSIZE,1)

color_lower = np.array([0, 43, 46])
color_upper = np.array([10, 255, 255])


def Color_Recongnize():
    
    while(1):
        # get a frame and show 获取视频帧并转成HSV格式, 利用cvtColor()将BGR格式转成HSV格式，参数为cv2.COLOR_BGR2HSV。
        ret, frame = cap.read()
        frame, color_name = get_color(frame)
        if len(color_name)==1:
            global color_lower
            global color_upper

            if color_name['name'] == 'yellow':
                color_lower = np.array([26, 43, 46])
                color_upper = np.array([34, 255, 255])
                
            elif color_name['name'] == 'red':
                color_lower = np.array([0, 43, 46])
                color_upper = np.array([10, 255, 255])
               
            elif  color_name['name'] == 'green':
                color_lower = np.array([35, 43, 46])
                color_upper = np.array([77, 255, 255])
              
            elif color_name['name'] == 'blue':
                color_lower=np.array([100, 43, 46])
                color_upper = np.array([124, 255, 255])

        cv2.imshow("img",frame)
        cv2.waitKey(1)



    cap.release()
    cv2.destroyAllWindows()

In [7]:
def color_recognize_1frame(cap):
    ret, frame = cap.read()
    frame, color_name = get_color(frame)

    if len(color_name)==1:
        print(color_name)
    return frame, color_name

In [30]:
def face_filter(faces):

        if len(faces) == 0: return None

        max_face = max(faces, key=lambda face: face[2] * face[3])
        (x, y, w, h) = max_face

        if w < 10 or h < 10: return None
        return max_face
    
def face_find_1frame(cap, face_detector):
    cap.read()
    ret,img = cap.read()
    
    img = cv2.resize(img, (640, 480))

    img = img.copy()

    gray = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)

    faces = face_detector.detectMultiScale(gray, scaleFactor=1.3, minNeighbors=5)
    point_x =-1
    point_y = -1
    if len(faces) != 0:
        face = face_filter(faces)

        (x, y, w, h) = face

        cv2.rectangle(img, (x, y), (x + w, y + h), (0, 255, 0), 4)
        point_x = x + w / 2
        point_y = y + h / 2

    return point_x,point_y,img


def clamp(x,minv,maxv):
    return max(min(x,maxv),minv)

def get_pos(x,y):
    
    rx = 10
    ry = 5

    cx = 320
    cy = 240

    target_x = -1*(x-cx)/rx
    target_y = -1*(y-cy)/ry

    target_x = clamp(target_x,-45,45)
    target_y = clamp(target_y,-25,45)

    middle_y = 45
    middle_x = 90

    _x = middle_x+target_x
    _y = middle_y+target_y

    print(f"({target_x}, {target_y})")
    print(f"({x}, {y})")
    return _x,_y

def go_s(x,y):
    s3range = 45

    _x,_y = get_pos(x,y)
    
    moves = [_x,_y,90,90-_y,90,80]
    Arm.Arm_serial_servo_write6_array(moves,1000)
    time.sleep(1)
    moves[2] = 90-s3range
    moves[3] = 90-_y+s3range
    Arm.Arm_serial_servo_write6_array(moves,1000)
    time.sleep(1)
    moves[2] = 90
    moves[3] = 90-_y
    Arm.Arm_serial_servo_write6_array(moves,1000)
    time.sleep(1)

    

In [13]:
go_s(120,280)

(-45, 20.0)


In [45]:
cap.release()

In [47]:
cap = cv2.VideoCapture(0)
cap.set(3, 480)
cap.set(4, 320)
cap.set(5, 15)  #设置帧率
cap.set(cv2.CAP_PROP_FOURCC, cv2.VideoWriter.fourcc('M', 'J', 'P', 'G'))
cap.set(cv2.CAP_PROP_BUFFERSIZE,1)
faceDetect = cv2.CascadeClassifier("haarcascade_frontalface_default.xml")
color_lower = np.array([0, 43, 46])
color_upper = np.array([10, 255, 255])
arm_reset()
try:
    while(1):

        f,color_name = color_recognize_1frame(cap)

        if len(color_name)==1:

            if color_name['name'] == 'yellow':
                Arm.Arm_RGB_set(50, 50, 0)
                arm_grip_and_pour(60,90,'left')
            elif color_name['name'] == 'blue':
                Arm.Arm_RGB_set(0, 0, 50)
                arm_grip_and_pour(120,60,'right')
            elif  color_name['name'] == 'green':
                Arm.Arm_RGB_set(0, 50, 0)
                time.sleep(2)
                _x, _y, img = face_find_1frame(cap,faceDetect)
                go_s(_x, _y)

            elif color_name['name'] == 'red':
                pass

            arm_reset()
        Arm.Arm_RGB_set(0, 0, 0)

        cv2.imshow("cap",f)
        cv2.waitKey(1)
except KeyboardInterrupt:
    cv2.destroyAllWindows()
    cap.release()
    

In [ ]:
target_servox=90
target_servoy=45
target_servoa=45

servo_3_mvrange =45

Arm.Arm_serial_servo_write6(target_servox,target_servoy,90,90-target_servoy,90,80,1000)
time.sleep(1)
Arm.Arm_serial_servo_write6(target_servox,target_servoy,90-servo_3_mvrange,90-target_servoy+servo_3_mvrange,90,80,1000)
time.sleep(1)
Arm.Arm_serial_servo_write6(target_servox,target_servoy,90,90-target_servoy,90,80,1000)
time.sleep(1)
#joints_0 = [target_servox, target_servoy, 90, 90-target_servoy, 90,80]
#Arm.Arm_serial_servo_write6_array(joints_0,1000)
#time.sleep(1)